# Bluebook for Bulldozers

We will be looking at the Blue Book for Bulldozers Kaggle Competition: "The goal of the contest is to predict the sale price of a particular piece of heavy equiment at auction based on it's usage, equipment type, and configuration. The data is sourced from auction result postings and includes information on usage and equipment configurations."

This dataset/competition has been chosen because of the closeness of the data to the realtime workplace.

Link here : https://www.kaggle.com/c/bluebook-for-bulldozers

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
from structured import *
import warnings
from sklearn.ensemble import RandomForestRegressor

from sklearn import metrics 

warnings.filterwarnings('ignore')

In [3]:
df_raw = pd.read_csv('data/Train.csv',low_memory=False,
                    parse_dates=["saledate"])
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3.0,2001,3486.0,High,2011-05-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3.0,2007,722.0,Medium,2009-07-23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_raw.saledate

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

In [5]:
#since the kaggle competition evaluation metric is the RMSLE(Root mean square log error)
df_raw.SalePrice=np.log(df_raw.SalePrice)

## Initial processing

you need to drop the target variable convert the categorical variables to numbers and then fit 

The dataset has both continous and categorical variables like the datetime thing etc which you can use.
And you need a piece of feature engineering to get info out of this

### 1. Dealing with dates

In [6]:
df_raw.saledate #datatype is datetime

0        2006-11-16
1        2004-03-26
2        2004-02-26
3        2011-05-19
4        2009-07-23
            ...    
401120   2011-11-02
401121   2011-11-02
401122   2011-11-02
401123   2011-10-25
401124   2011-10-25
Name: saledate, Length: 401125, dtype: datetime64[ns]

The **add_datepart** method extracts particular date fields from a complete datetime for the purpose of constructing categoricals. You should always consider this feature extraction step when working with date-time. Without expanding your date-time into these additional fields, you can't capture any trend/cyclical behavior as a function of time at any of these granularities.

In [7]:
add_datepart(df_raw, 'saledate')

In [8]:
df_raw.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1248307200


### 2. Convert strings to numbers for pandas

In [9]:
df_raw.columns

Index(['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'datasource',
       'auctioneerID', 'YearMade', 'MachineHoursCurrentMeter', 'UsageBand',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'saleYear', 'saleMonth',
       'saleWeek', 'saleDay', 'saleDayofweek', 'saleDayofyear',


In [10]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 65 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null float64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null object
fiModelDesc                 401125 non-null object
fiBaseModel                 401125 non-null object
fiSecondaryDesc             263934 non-null object
fiModelSeries               56908 non-null object
fiModelDescriptor           71919 non-null object
ProductSize                 190350 non-null object
fiProductClassDesc          401125 non-null object
state                       401125 non-null object
ProductGroup               

The categorical variables are currently stored as strings, which is inefficient, and doesn't provide the numeric coding required for a random forest. Therefore we call **train_cats** to convert strings to pandas categories.

In [11]:
train_cats(df_raw) #converts most of these objects into categories

In [12]:
df_raw.info() #how most objects have been turned to category

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401125 entries, 0 to 401124
Data columns (total 65 columns):
SalesID                     401125 non-null int64
SalePrice                   401125 non-null float64
MachineID                   401125 non-null int64
ModelID                     401125 non-null int64
datasource                  401125 non-null int64
auctioneerID                380989 non-null float64
YearMade                    401125 non-null int64
MachineHoursCurrentMeter    142765 non-null float64
UsageBand                   69639 non-null category
fiModelDesc                 401125 non-null category
fiBaseModel                 401125 non-null category
fiSecondaryDesc             263934 non-null category
fiModelSeries               56908 non-null category
fiModelDescriptor           71919 non-null category
ProductSize                 190350 non-null category
fiProductClassDesc          401125 non-null category
state                       401125 non-null category
ProductGr

**Note** :Category is a pandas datatype

In [13]:
df_raw.UsageBand

0            Low
1            Low
2           High
3           High
4         Medium
           ...  
401120       NaN
401121       NaN
401122       NaN
401123       NaN
401124       NaN
Name: UsageBand, Length: 401125, dtype: category
Categories (3, object): [High < Low < Medium]

In [14]:
df_raw.UsageBand.cat.categories #gives you the categories for the usage band feature

Index(['High', 'Low', 'Medium'], dtype='object')

To make things easier for the random forest, we rearrange the categories in the UsageBand feature to make more sense to split on

In [15]:
df_raw.UsageBand.cat.set_categories(['High', 'Medium', 'Low'], ordered=True, inplace=True)
#order it so the splitting gets the maximum benifit from it 

In [16]:
df_raw.UsageBand.cat.categories

Index(['High', 'Medium', 'Low'], dtype='object')

Normally, pandas will continue displaying the text categories, while treating them as numerical data internally. Optionally, we can replace the text categories with numbers, which will make this variable non-categorical, like so:.

In [17]:
df_raw.UsageBand.cat.codes

0         2
1         2
2         0
3         0
4         1
         ..
401120   -1
401121   -1
401122   -1
401123   -1
401124   -1
Length: 401125, dtype: int8

In [18]:
df_raw.head() #usage band still says high or low but behind the scenes they've been made into numbers

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,fiModelDesc,...,saleDay,saleDayofweek,saleDayofyear,saleIs_month_end,saleIs_month_start,saleIs_quarter_end,saleIs_quarter_start,saleIs_year_end,saleIs_year_start,saleElapsed
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,16,3,320,False,False,False,False,False,False,1163635200
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,26,4,86,False,False,False,False,False,False,1080259200
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,26,3,57,False,False,False,False,False,False,1077753600
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,19,3,139,False,False,False,False,False,False,1305763200
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,23,3,204,False,False,False,False,False,False,1248307200


## pre-processing

In [19]:
df_raw.to_feather(('tmp/bulldozers-raw'))

In [20]:
df_raw = pd.read_feather('tmp/bulldozers-raw')

 what proc_df does :
 proc_df takes a data frame df and splits off the response variable, and
 changes the df into an entirely numeric dataframe. For each column of df 
 which is not in skip_flds nor in ignore_flds, na values are replaced by the
median value of the column.

 1. fix_missing - Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.
 2. scale_vars(if needed)
 3. numericalize - Changes the column col from a categorical type to it's integer codes.

In [21]:
df, y, nas = proc_df(df_raw, 'SalePrice')

Creating a validation set

In [22]:
def split_vals(a,n): 
    return a[:n].copy(), a[n:].copy()

n_valid = 12000  # same as Kaggle's test set size
n_trn = len(df)-n_valid
# raw_train, raw_valid = split_vals(df_raw, n_trn)
X_train, X_valid = split_vals(df, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, y_train.shape, X_valid.shape

((389125, 66), (389125,), (12000, 66))

In [23]:
import math
#let's track the metrics we're interested in 
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

### Modelling

In [24]:
m = RandomForestRegressor() 
%time m.fit(X_train, y_train)
print_score(m) #training rmse, valid rmse, training accuracy and validation accuracy respectively

Wall time: 2min 3s
[0.09023557179577871, 0.2512440691098927, 0.9829827109605074, 0.8872699729798288]


2 mins is too long and anything more than 10 secs will slow down the iteration process. so use proc_df to reduce the size of the training set. Proc_df has a subset attribute that handles it

In [25]:
len(df_raw)

401125

In [26]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice', subset=50_000) 
X_train, _ = split_vals(df_trn, 40_000) 
y_train, _ = split_vals(y_trn, 40_000) 

In [27]:
m = RandomForestRegressor()
%time m.fit(X_train, y_train)
print_score(m)

Wall time: 9.74 s
[0.1068789642277177, 0.30717602923580867, 0.9761229343515654, 0.8314911874512791]


Use that subset which takes around 8-10 seconds to compute

note that as you increase the size of the subset, the accuracy of the model increases meaning you can try out the hyperparameters here, and then go back to the bigger dataset after you find the best ones

Each tree is stored in estimators_ so run the validation set through each tree 
So for every row you have 1 prediction per tree, so 12000 pedictions per tree and there are 10(will change to 100 as default) trees

In [28]:
preds = np.stack([t.predict(X_valid) for t in m.estimators_])
preds[:,0], np.mean(preds[:,0]), y_valid[0]

(array([9.51044496, 9.43348392, 9.04782144, 9.23503298, 9.35010231,
        9.35010231, 9.39266193, 9.10497986, 9.47270464, 9.51044496]),
 9.340777932942324,
 9.104979856318357)

In [29]:
preds.shape

(10, 12000)

In [31]:
m = RandomForestRegressor(n_estimators=20, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m) #prev was 81

[0.09700609687649143, 0.2999686713582601, 0.9803304446576905, 0.8393059586980616]


In [35]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.0924182896010766, 0.2951094705873538, 0.9821469535818758, 0.844469965620507]


In [37]:
m = RandomForestRegressor(n_estimators=50, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.09137264377102547, 0.2936115902354002, 0.9825486566732404, 0.8460447992146723]


In [38]:
m = RandomForestRegressor(n_estimators=75, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.08957876461099411, 0.29279644987228853, 0.9832271594557018, 0.8468984500874728]


In [39]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.08904858059303238, 0.29145626113542555, 0.9834251165436692, 0.8482967961825842]


In [41]:
m = RandomForestRegressor(n_estimators=125, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.08862690034063811, 0.2909901749031515, 0.9835817221277294, 0.8487816046994242]


In [53]:
m = RandomForestRegressor(n_estimators=160, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.08805881668151745, 0.2905616305159194, 0.9837915244161933, 0.849226678667631]


In [52]:
m = RandomForestRegressor(n_estimators=170, n_jobs=-1)
m.fit(X_train, y_train)
print_score(m)

[0.08811704662434934, 0.2918660352674021, 0.9837700812331511, 0.8478699208200838]


Score decreases at 170. So let's revert to 160

### Out-of-bag (OOB) score

In [57]:
m = RandomForestRegressor(n_estimators=160, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m) #final output is oob error

[0.08807980149411163, 0.29155255604149743, 0.9837837983902745, 0.8481965364802777, 0.8811346312656196]


This shows that our validation set time difference is making an impact, as is model over-fitting.

## Reducing over-fitting

It turns out that one of the easiest ways to avoid over-fitting is also one of the best ways to speed up analysis: subsampling. Let's return to using our full dataset, so that we can demonstrate the impact of this technique.

In [58]:
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)

The basic idea is this: rather than limit the total amount of data that our model can access, let's instead limit it to a different random subset per tree. That way, given enough trees, the model can still see all the data, but for each individual tree it'll be just as fast as if we had cut down our dataset as before.

In [59]:
set_rf_samples(20000)

reset_rf_samples() turns it off

In [60]:
m = RandomForestRegressor(n_jobs=-1, oob_score=True) #doesn't work but may now 
%time m.fit(X_train, y_train)
print_score(m)

Wall time: 4.54 s
[0.23989523320803696, 0.28014618699769245, 0.8797242529453847, 0.859842156888388, 0.8672439881136268]


In [61]:
m = RandomForestRegressor(n_estimators=40, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.22712928618526948, 0.26416908457726207, 0.892184525613748, 0.8753730529222455, 0.8806606774740937]


In [62]:
m = RandomForestRegressor(n_estimators=50, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.22656111079717642, 0.26404080603985236, 0.8927232626468637, 0.8754940593673689, 0.8812096471032818]


In [63]:
m = RandomForestRegressor(n_estimators=75, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.22535893819709282, 0.26192227657275435, 0.8938587002165946, 0.8774839891184583, 0.8824561999706062]


In [64]:
m = RandomForestRegressor(n_estimators=100, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.22465142900219381, 0.2597737265453955, 0.894524110385453, 0.8794857442652273, 0.8831718818263348]


In [65]:
m = RandomForestRegressor(n_estimators=120, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.22411941157889298, 0.2594211063387646, 0.8950230928823695, 0.8798126974192385, 0.8837073821409354]


In [66]:
m = RandomForestRegressor(n_estimators=140, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.2240485058356404, 0.2588953717507494, 0.8950895064795538, 0.8802993393651891, 0.8837925091963417]


In [67]:
set_rf_samples(50000)

In [68]:
m = RandomForestRegressor(n_estimators=150, min_samples_leaf=3, max_features=0.5, n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

[0.2034794779759703, 0.24663159234337148, 0.9134681366742609, 0.891371098503317, 0.8968496197084446]


With the model now at near 90% accuracy, we can dig deep in for further insights (as in the next notebook)